<a href="https://colab.research.google.com/github/emmaenglishwilkins/trading-algo-set-up/blob/main/autotradeSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

This is a working automated trading bot with basic strategy, placing paper market orders on a single symbol (stock) for now...

## Imports
* using alpaca trading api
* using python libraries such as pandas, numpy seaborn, time, requests, math,etc
* google sheets api for logging


## Configuration
* secret key and alpaca trading api key to connect to platform
* authentication to google sheets to write to




### Imports

In [1]:
!pip install alpaca-trade-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 4.7 MB/s eta 0:00:00
  Created wheel for msgpack: filename=msgpack-1.0.3-cp311-cp311-linux_x86_64.whl size=15688 sha256=36e090bce97f2804b4ede43a3edcc71906d9d058f03fb036a875b692f56fd339
  Stored in directory: /root/.cache/pip/wheels/f6/35/da/ed9b26b510235e00e3a3c3bab7bad97b59214729662255ab3d
Successfully built msgpack
  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.1.0
    Uninstalling msgpack-1.1.0:
      Successfully uninstalled msgpack-1.1.0
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling 

In [2]:
!pip install alpaca.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 4.2 MB/s eta 0:00:00


In [3]:
!pip install --upgrade gspread gspread_dataframe
!pip install gspread
!pip install oauth2client

In [4]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import requests
import time
import math

import alpaca_trade_api as tradeapi

# import backtrader as bt

### Configuration

connects to alpaca account and gets variable for trading client - prints inital account number and buying power of the account - part of general set up

In [8]:
# configuration
from google.colab import userdata

ALPACA_API_KEY = userdata.get('ALPACA_API_KEY')
ALPACA_SECRET_KEY = userdata.get('ALPACA_SECRET_KEY')
ALPACA_BASE_URL = 'https://paper-api.alpaca.markets/v2'

api = tradeapi.REST(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPACA_BASE_URL, 'v2')
session = requests.Session()

In [9]:
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [10]:
from alpaca.trading.client import TradingClient

trading_client = TradingClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

print(trading_client.get_account().account_number)
print(trading_client.get_account().buying_power)

PA3FTDD94XAH
199507.32


In [12]:
import gspread_dataframe as gd
import pandas as pd

# --- Daily Sheet Handling ---
def get_daily_sheet():
    today_str = datetime.now().strftime('%Y-%m-%d')
    try:
        sh = gc.open('TradingBotLogs')
        try:
            worksheet = sh.worksheet(today_str)
        except gspread.exceptions.WorksheetNotFound:
            worksheet = sh.add_worksheet(title=today_str, rows="1000", cols="10")
            worksheet.update('A1', [['Timestamp', 'Action', 'Symbol', 'Price', 'Quantity']])
    except Exception as e:
        print(f"🔥 Error setting up sheet: {e}")
        worksheet = None
    return worksheet

# --- Logging a Trade ---
def log_trade(action, symbol, price, qty):
    sheet = get_daily_sheet()
    if sheet:
        now = pd.Timestamp.now(tz='America/New_York')  # NYSE timezone
        row = [[now.strftime('%Y-%m-%d %H:%M:%S'), action, symbol, price, qty]]
        sheet.append_rows(row)
        print(f"📋 Logged Trade: {row}")
    else:
        print("❌ Sheet not available!")


### Historical Data

though not all market trends will be the same as past historical data it can be good to capture it to understand processing -- helping with trading without emotion using tend (almanac of previous pricing)

In [13]:
# get historical data
from alpaca.data import StockHistoricalDataClient, StockTradesRequest
from datetime import datetime

data_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

request_params = StockTradesRequest(
    symbol_or_symbols = "AAPL",             # apple
    start= datetime(2025, 3, 17, 14, 30),
    end= datetime(2025, 3, 17, 15, 30)
)

trades = data_client.get_stock_trades(request_params)

# print(trades) --

### Functionality

* regualr market order (functional)
* limit order (functional)

### Strategy
* get moving averages
* generate signal
* get current price (refresh on the minute)

In [14]:
# placing order reg_market_order limit_order
# side = OrderSide.BUY or OrderSide.SELL cleaner - can be reused
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.requests import LimitOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

def reg_market_order(symbol, qty, side):
  # regular market order
  market_order_data = MarketOrderRequest(
      symbol=symbol,
      qty=qty,
      side=side,
      time_in_force=TimeInForce.DAY
  )
  market_order_data = trading_client.submit_order(market_order_data)
  return market_order_data

def limit_order(symbol, qty, limit_price, side):
  # Shorting --?? / Limit order
  limit_order_data = MarketOrderRequest(
      symbol=symbol,
      qty=qty,
      side=side,
      time_in_force=TimeInForce.DAY,
      limit_price=limit_price
  )
  limit_order_response = trading_client.submit_order(limit_order_data)
  return limit_order_response

# print(market_order_data)

In [15]:
# strategy logic
import pandas as pd
import time

def get_moving_averages(symbol, short_window, long_window):
    bars = trading_client.get_bars(symbol, timeframe='1Min', limit=long_window).df
    bars = bars[bars['symbol'] == symbol]

    bars['short_ma'] = bars['close'].rolling(window=short_window).mean()
    bars['long_ma'] = bars['close'].rolling(window=long_window).mean()

    return bars

def generate_signal(df):
    if df['short_ma'].iloc[-1] > df['long_ma'].iloc[-1] and df['short_ma'].iloc[-2] <= df['long_ma'].iloc[-2]:
        return 'buy'
    elif df['short_ma'].iloc[-1] < df['long_ma'].iloc[-1] and df['short_ma'].iloc[-2] >= df['long_ma'].iloc[-2]:
        return 'sell'
    else:
        return 'hold'

def get_current_price(symbol):
    barset = trading_client.get_bars(symbol, timeframe='1Min', limit=1).df
    return barset['close'].iloc[-1]

### Trading Loop

currently only uses one Symbol (apple) --> waits 60sec between infinite loop

* added Stop-Loss and Take-Profit
* added trailing stop-loss

** TODO: add trailing take-profit
* first want to implement trading multiple stocks
* and add automation

In [17]:
# Bot settings (global variables)
BUDGET = 10000  # Total dollars you want to use
TRAILING_PERCENT = 3  # Trailing Stop Loss % (example: 3%)
SYMBOLS = ['AAPL', 'MSFT', 'TSLA', 'PLTR', 'AMD', 'GOOGL', 'SOFI']  # Your stock list
MAX_POSITIONS = len(SYMBOLS)  # We spread budget evenly across symbols
MARKET_OPEN = "09:30"
MARKET_CLOSE = "16:00"
TRADE_AMOUNT = BUDGET // MAX_POSITIONS  # Even distribution of budget across stocks
qty = TRADE_AMOUNT  # How much of each stock to buy based on your budget

In [18]:
def is_market_open():
    now = datetime.now().strftime("%H:%M")
    return MARKET_OPEN <= now <= MARKET_CLOSE

In [19]:
import time
import random
from datetime import datetime

short_window = 20
long_window = 50

# State per stock
stock_data = {symbol: {'position': None, 'buy_price': None, 'stop_loss_price': None} for symbol in SYMBOLS}

def is_market_open():
    now = datetime.now().strftime('%H:%M')
    return MARKET_OPEN <= now <= MARKET_CLOSE

# --- Main Trading Loop ---
while True:
    try:
        if not is_market_open():
            print("Market closed. Sleeping...")
            time.sleep(300)  # sleep for 5 min when market closed
            continue

        for symbol in SYMBOLS:
            # Fetch the stock data and signals (this is your own method)
            df = get_moving_averages(symbol, short_window, long_window)
            signal = generate_signal(df)
            current_price = get_current_price(symbol)

            data = stock_data[symbol]

            if data['position'] is None:
                if signal == 'buy':
                    # Buy stock
                    order = reg_market_order(symbol, qty, OrderSide.BUY)
                    data['buy_price'] = float(current_price)
                    data['stop_loss_price'] = data['buy_price'] * (1 - (TRAILING_PERCENT / 100))
                    data['position'] = 'long'

                    log_trade('BUY', symbol, data['buy_price'], qty)
                    print(f"✅ Bought {symbol} at ${data['buy_price']:.2f} | Stop-loss: ${data['stop_loss_price']:.2f}")
                else:
                    print(f"[{symbol}] Waiting to Buy. Signal: {signal}")

            else:
                # Update Trailing Stop
                potential_new_stop = current_price * (1 - (TRAILING_PERCENT / 100))
                if potential_new_stop > data['stop_loss_price']:
                    data['stop_loss_price'] = potential_new_stop
                    print(f"[{symbol}] Trailing Stop moved up to ${data['stop_loss_price']:.2f}")

                # Check for Stop-Loss Sell
                if current_price <= data['stop_loss_price']:
                    order = reg_market_order(symbol, qty, OrderSide.SELL)

                    log_trade('SELL', symbol, current_price, qty)
                    print(f"🚨 Sold {symbol} at ${current_price:.2f} due to stop-loss trigger.")

                    # Reset state
                    stock_data[symbol] = {'position': None, 'buy_price': None, 'stop_loss_price': None}
                else:
                    print(f"[{symbol}] Holding... Current: ${current_price:.2f} | Stop-loss: ${data['stop_loss_price']:.2f}")

    except Exception as e:
        print(f"🔥 Error in main loop: {e}")

    # Random sleep between 55 and 65 seconds to mimic human behavior
    sleep_time = random.randint(55, 65)
    print(f"Sleeping {sleep_time} seconds...\n")
    time.sleep(sleep_time)


Market closed. Sleeping...


KeyboardInterrupt: 

### Managing account
* get all orders
* bulk operation for getting all orders
* streaming stock market data

In [ ]:
# # get, cancel and managing orders -- managing orders in queue
# # or else they live there forever ??
# from alpaca.trading.requests import GetOrdersRequest
# from alpaca.trading.enums import OrderSide, QueryOrderStatus

# request_params = GetOrdersRequest(
#     status=QueryOrderStatus.OPEN,
# )

# orders = trading_client.get_orders(request_params)

# for order in orders:
#   print(order.id)
#   trading_client.cancel_order_by_id(order.id)

In [ ]:
# # updating all -- bulk operation

# positions = trading_client.get_all_positions()
# # print(positions)
# for position in positions:
#   print(position.symbol, position.current_price)
#   # increase set orders
#   # modify subset of postions (or all in this instance)

# # completly liquidate account
# # boolean value is for canceling orders as well
# # trading_client.close_all_positions(True)

In [ ]:
# # streaming stocks real time stock market trade data
# from alpaca.data.live import StockDataStream
# from alpaca.data.live import CryptoDataStream

# stream = StockDataStream(ALPACA_API_KEY, ALPACA_SECRET_KEY)

# async def handle_trade(data):
#   print(data)

# stream.subscribe_trades(handle_trade, "APPL")
# stream.run()

# async def handle_quote(data):
#   print(data)

# stream.subscribe_quotes(handle_trade, "APPL") # what you can buy stock for at every second?
# stream.run()

In [ ]:
# def get_market_times():
#     nyc = timezone('America/New_York')
#     today = datetime.now(nyc).date().isoformat()
#     calendar = api.get_calendar(start=today, end=today)[0]
#     open_time = calendar.open.astimezone(nyc)
#     close_time = calendar.close.astimezone(nyc)
#     return open_time, close_time

# def wait_for_market_open(open_time):
#     now = datetime.now(timezone('America/New_York'))
#     while now < open_time:
#         print("Waiting for market to open...")
#         time.sleep(30)
#         now = datetime.now(timezone('America/New_York'))

In [ ]:
# # risk management parameters
# MIN_SHARE_PRICE = 2.0
# MAX_SHARE_PRICE = 13.0
# MIN_DOLLAR_VOLUME = 500000
# DEFAULT_STOP_LOSS = 0.95
# RISK_PER_TRADE = 0.001

In [ ]:
# # data
# def get_tradable_symbols():
#   all_assets = api.list_assets()
#   tradable_symbols = [asset.symbol for asset in all_assets if asset.tradable and asset.status == 'active']
#   return tradable_symbols

# def get_recent_minute_history(symbol, limit=1000):
#   try:
#     barset = api.get_bars(symbol, tradeapi.TimeFrame.Minute, limit=limit).df
#     return barset
#   except Exception as e:
#     print(f"Error fetching data for {symbol}: {e}")
#     return None

In [ ]:
# def main():
#     symbols = get_tradable_symbols()
#     open_time, close_time = get_market_times()
#     wait_for_market_open(open_time)

#     print(f"Market is open! Evaluating symbols...")
#     portfolio_value = float(api.get_account().portfolio_value)

#     for symbol in symbols:
#         barset = get_recent_minute_history(symbol)
#         strategy(symbol, barset, portfolio_value)

In [ ]:
# class TestStrategy(bt.Strategy):
#   def __init__(self):
#     pass
#   def next(self):
#     if not self.position:
#       self.buy()
#     else:
#       self.close()

# cerebro = bt.Cerebro()
# data = bt.feeds.
# cerebro.adddata(data)
# cerebro.addstrategy(TestStrategy)
# cerebro.run()
# cerebro.plot()